In [42]:
import lipoPredict.lipoPredict as lp
from Bio import SeqIO, SeqRecord

In [43]:
lr_roary_fa = "output/lr_roary/LR_pan_genome_reference.fa"
sr_roary_fa = "output/sr_roary/SR_pan_genome_reference.fa"

In [44]:
lr_roary = SeqIO.parse(lr_roary_fa, "fasta")
sr_roary = SeqIO.parse(sr_roary_fa, "fasta")

In [45]:
lr_fixed = []
sr_fixed = []
for record in lr_roary:
    sampleid = record.description.split(' ')[0]
    intermediate_name = record.description.split(' ')[1::]
    intermediate_name = "-".join(intermediate_name)
    fixed_name = f"{sampleid}__{intermediate_name}"
    seq = record.seq
    seqrecord = SeqRecord.SeqRecord(seq = seq, id=fixed_name, description="")
    lr_fixed.append(seqrecord)


In [46]:
for record in sr_roary:
    sampleid = record.description.split(' ')[0]
    intermediate_name = record.description.split(' ')[1::]
    intermediate_name = "-".join(intermediate_name)
    fixed_name = f"{sampleid}__{intermediate_name}"
    seq = record.seq
    seqrecord = SeqRecord.SeqRecord(seq = seq, id=fixed_name, description="")
    sr_fixed.append(seqrecord)


In [47]:
SeqIO.write(sr_fixed, f"output/sr_roary/sr_pangenome_ref_FIXED.fa", "fasta")
SeqIO.write(lr_fixed, f"output/lr_roary/lr_pangenome_ref_FIXED.fa", "fasta")

3921

In [48]:
# ok now let's get the matrix given the group matrix output by roary.
lr_clustered = "../longread_analysis/v4/longread_paired_v4/clustered_proteins"
sr_clustered = "../longread_analysis/v4/shortread_paired_v4/clustered_proteins"


In [80]:
fixed_lr_clustered = []
fixed_sr_clustered = []

with open(lr_clustered) as f:
    for line in f:
        fixed_lr_clustered.append(line.replace(":","\t"))

with open(sr_clustered) as f:
    for line in f:
        fixed_sr_clustered.append(line.replace(":","\t"))

In [81]:
with open("output/lr_roary/lr_clustered_proteins_FIXED.txt", "w") as f:
    f.writelines(fixed_lr_clustered)

with open("output/sr_roary/sr_clustered_proteins_FIXED.txt", "w") as f:
    f.writelines(fixed_sr_clustered)

In [82]:
import pandas

In [90]:
df = pandas.read_csv("output/lr_roary/lr_clustered_proteins_FIXED.txt", sep="\t", header=None)
df.columns = ['GeneGroups'] + [i for i in range(1, len(df.columns))] # this pulls columns from row 0 through to the end of the dataframe for the first column.
unique_genomes = set()
for index, row in df.iterrows():
    for genome in row[1::]:
        unique_genomes.add(genome)
print(len(unique_genomes))

68136
                 GeneGroups              1             2             3  \
0               Lipoprotein   PHIDNK_04785  KBFCPJ_06370  EBACPA_05215   
1     Putative antigen, P35   HOFFII_06695  JKOCPB_06080  KKDEEG_04585   
2                      lptF   DMPBEC_04070  AIHKLI_04085  LNEFEP_04075   
3                group_3281   DMPBEC_04075  AIHKLI_04090  LNEFEP_04080   
4                      htpG   DMPBEC_02785  AIHKLI_02800  LNEFEP_02790   
...                     ...            ...           ...           ...   
3916             group_2276   EBOALB_05460           NaN           NaN   
3917              group_704   JOOONJ_05885           NaN           NaN   
3918             group_1306   HMIDLJ_07370           NaN           NaN   
3919             group_1689   DMPBEC_04680           NaN           NaN   
3920             group_1218   KBFCPJ_03165           NaN           NaN   

                 4             5             6             7             8  \
0     FOOKID_05245  HOFFII_

In [88]:
print(binary_matrix)

                       1   2   3   4   5   6   7   8   9   10  ...  65  66  \
0                                                              ...           
Lipoprotein             0   0   0   0   0   0   0   0   0   0  ...   0   0   
Putative antigen, P35   0   0   0   0   0   0   0   0   0   0  ...   0   0   
lptF                    0   0   0   0   0   0   0   0   0   0  ...   0   0   
group_3281              0   0   0   0   0   0   0   0   0   0  ...   0   0   
htpG                    0   0   0   0   0   0   0   0   0   0  ...   0   0   
...                    ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..   
group_2276              0   0   0   0   0   0   0   0   0   0  ...   0   0   
group_704               0   0   0   0   0   0   0   0   0   0  ...   0   0   
group_1306              0   0   0   0   0   0   0   0   0   0  ...   0   0   
group_1689              0   0   0   0   0   0   0   0   0   0  ...   0   0   
group_1218              0   0   0   0   0   0   0   0   0   0  .

In [91]:
lr_lipoP_results = "output/fixed_output/lr_pangenome_ref_FIXED_lipoP_putatives.tsv"
sr_lipoP_results = "output/fixed_output/sr_pangenome_ref_FIXED_lipoP_putatives.tsv"

In [94]:
lr_lipoP_df = pandas.read_csv(lr_lipoP_results, sep="\t", header=None)
sr_lipoP_df = pandas.read_csv(sr_lipoP_results, sep="\t", header=None)

In [96]:
lr_lipo_groups = set(lr_lipoP_df[0])
sr_lipo_groups = set(sr_lipoP_df[0])

In [97]:
print(len(lr_lipo_groups))

514


In [98]:
print(len(sr_lipo_groups))

613


In [101]:
common_lipo_groups = lr_lipo_groups.union(sr_lipo_groups)
unique_lipo_groups = lr_lipo_groups.symmetric_difference(sr_lipo_groups)

In [102]:
print(len(unique_lipo_groups))

1127


### Let's pull the groups from our list of lipoproteins and then get the list of BB names!
fortunately I've already made a mapping tsv. let's import that.

In [115]:
lr_group2bb_file = "../group2bb/output/lr_groupID_to_BBgene.tsv"
sr_group2bb_file = "../group2bb/output/sr_groupID_to_BBgene.tsv"

In [116]:
lr_g2bb= pandas.read_csv(lr_group2bb_file, sep="\t", header=None)
sr_g2bb= pandas.read_csv(sr_group2bb_file, sep="\t", header=None)

In [119]:
lr_g2bb_dict = {}
sr_g2bb_dict = {}
lr_mapping_dict = dict(zip(lr_g2bb.iloc[:, 0], lr_g2bb.iloc[:, 1]))
sr_mapping_dict = dict(zip(sr_g2bb.iloc[:, 0], sr_g2bb.iloc[:, 1]))

In [124]:
lr_BB_groups = []
sr_BB_groups = []

for group in lr_lipo_groups:
    group = group.split("__")[0]
    bb_gene = lr_mapping_dict.get(group)
    print(f"{group} <-> {bb_gene}")
    lr_BB_groups.append(bb_gene)

for group in sr_lipo_groups:
    group = group.split("__")[0]
    bb_gene = sr_mapping_dict.get(group)
    print(f"{group} <-> {bb_gene}")
    sr_BB_groups.append(bb_gene)


PHIDNK_04850 <-> None
LGOLAD_05900 <-> group_1341
DMPBEC_04680 <-> group_1689
ELJNOB_05975 <-> group_3249
DMPBEC_03280 <-> secD
PNPFKP_06715 <-> group_2356
AIHKLI_07150 <-> group_1547
LNEFEP_05800 <-> None
AIHKLI_05020 <-> group_2580
CBONDI_06375 <-> group_2184
DMPBEC_02965 <-> rnfE
DMPBEC_02205 <-> fbaA
AIHKLI_04870 <-> group_1888
ELJNOB_05400 <-> group_2018
DMPBEC_04405 <-> group_1684
DMPBEC_03170 <-> hypothetical protein
KBFCPJ_04980 <-> group_2309
DMPBEC_03815 <-> group_3753
FMOGGM_04485 <-> group_3236
DMPBEC_05725 <-> Borrelia ORF-A
DMPBEC_01650 <-> P26
KBFCPJ_06430 <-> group_2342
AIHKLI_05905 <-> erpC
DMPBEC_04050 <-> truB
AIHKLI_04885 <-> group_1890
DMPBEC_01625 <-> oppA
ECDEAP_04970 <-> group_1915
JFIHAH_04855 <-> group_3234
JKOCPB_05560 <-> group_2132
FOFEEG_05710 <-> group_2225
HKCNAF_04585 <-> group_1018
AIHKLI_04520 <-> group_861
AIHKLI_04755 <-> group_1500
AIHKLI_05665 <-> erpD
DMPBEC_06105 <-> mlpI
HILFBG_04345 <-> group_2249
DMPBEC_01560 <-> group_2773
AEPIPJ_06060 <-> g

In [125]:
import pickle
with open("output/fixed_output/lr_BB_groups.pkl", "wb") as f:
    pickle.dump(lr_BB_groups, f)
with open("output/fixed_output/sr_BB_groups.pkl", "wb") as f:
    pickle.dump(sr_BB_groups, f)